In [4]:
import json
import random
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
words=[]
classes=[]
documents=[]
ignore_words = ['!', '?']
data_file = open('intents.json').read()
intents= json.loads(data_file)

In [17]:
lemmatizer= WordNetLemmatizer()

In [21]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w= nltk.word_tokenize(pattern)
        words.extend(w)
        #to corpus
        documents.append((w, intent['tag']))
        #add to our classess list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    

In [10]:
#lemmatize, lower each word and remove duplicate
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words= sorted(list(set(classes)))

In [11]:
len(documents)

47

In [12]:
print(len(classes), "classes", classes)

9 classes ['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [13]:
print(len(words), "unique lemmatized words", words)

9 unique lemmatized words ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [16]:
pickle.dump(words,open("words.pkl", 'wb'))

In [17]:
pickle.dump(classes, open('classes.pkl', 'wb'))

In [34]:
training= []
output_empty= [0]* len(classes)

#training set of words for each sentence
for doc in documents:
    bag=[]
    pattern_words = doc[0]
    #lemmatize each word-create base eord, in attempt to represent
    pattern_words= [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    #create bag of words array with 1 , if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
    #outputis 0 for each tag and 1 for each pattern
    output_row= list(output_empty)
    output_row[classes.index(doc[1])]= 1
    training.append([bag, output_row])
    
    
#shuffle our feature and turn into np.array
random.shuffle(training)
training= np.array(training)
#create train and test list
train_X= list(training[:, 0])
train_Y = list(training[:, 1])
print("training data created")

training data created


In [33]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD 

In [45]:
model= Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation='softmax'))


#compile model, stochastic gradient descent with nesterov accelerated gradient
sgd= SGD(lr=0.01, decay= 1e-6, momentum= 0.9, nesterov= True)
model.compile(loss= 'categorical_crossentropy', optimizer= sgd, metrics=['accuracy'])

#fitting and saving
hist = model.fit(np.array(train_X), np.array(train_Y), epochs= 200, batch_size=5, verbose=1)


Epoch 1/200
9/9 [==============================] - 1s 72ms/step - loss: 2.1897 - acc: 0.4444
Epoch 2/200
9/9 [==============================] - 0s 554us/step - loss: 2.1429 - acc: 1.0000
Epoch 3/200
9/9 [==============================] - 0s 443us/step - loss: 2.0723 - acc: 1.0000
Epoch 4/200
9/9 [==============================] - 0s 553us/step - loss: 1.9835 - acc: 1.0000
Epoch 5/200
9/9 [==============================] - 0s 448us/step - loss: 1.8814 - acc: 1.0000
Epoch 6/200
9/9 [==============================] - 0s 443us/step - loss: 1.7701 - acc: 1.0000
Epoch 7/200
9/9 [==============================] - 0s 554us/step - loss: 1.6533 - acc: 1.0000
Epoch 8/200
9/9 [==============================] - 0s 562us/step - loss: 1.5342 - acc: 1.0000
Epoch 9/200
9/9 [==============================] - 0s 440us/step - loss: 1.4153 - acc: 1.0000
Epoch 10/200
9/9 [==============================] - 0s 554us/step - loss: 1.2990 - acc: 1.0000
Epoch 11/200
9/9 [==============================] - 0s 555us

In [46]:
model.save('chatbot_model.h5', hist)
print('model_saved')

model_saved


In [20]:
import tkinter
from tkinter import *

import json

from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
lemmatizer= WordNetLemmatizer()

def clean_up_sentence(sentence):
    #tokenize the pattern split words into array
    sentence_words=nltk.word_tokenize(sentence)
    #stem words and -create short form for word
    sentence_words=[lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    #tokenize pttern
    sentence_words= clean_up_sentence(sentence)
    #bag of words-matrix of N words, vocabulary
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w==s:
                #assign 1 if current word is in the vocabulary position
                bag[i]=1
                if show_details:
                    print("found in bag: %s" %w )
    return(np.array(bag))

def predict_class(sentence, model):
    #filter out prediction
    p= bow(sentence, words, show_details=False)
    res= model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = chatbot_response(msg)
            ChatLog.insert(END, "Bot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
        
base = Tk()
base.title("ChatBot by Girijeshwar")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)



#create chat window
ChatLog = Text(base, bd=0, bg="white", height= "8", width= "50", font="Arial",)
ChatLog.config(state= DISABLED)

scrollbar= Scrollbar(base, command= ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand']=scrollbar.set

#create button to send msg
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)


base.mainloop()